In [1]:
import os
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import random
from torchvision import datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.amp import GradScaler, autocast
import os
import random

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

class CustomDataset(Dataset):
    def __init__(self, image_dir, labels, transform=None):
        """
        Args:
            image_dir (str): Path to the directory containing saved images.
            labels (list): List of labels corresponding to the images.
            transform (callable, optional): Transformation to apply to the images.
        """
        self.image_paths = sorted([os.path.join(image_dir, img) for img in os.listdir(image_dir)])
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

dataset = datasets.Imagenette(root='/home/j597s263/scratch/j597s263/Datasets/imagenette', download=False, transform=transform)

random.seed(42)
indices = list(range(len(dataset)))
random.shuffle(indices)
train_indices = indices[:7568]
test_indices = indices[7568:8522]

train_labels = [dataset[i][1] for i in train_indices]

output_dir = "/home/j597s263/scratch/j597s263/Datasets/Defense/ConvImag_Shap"

defense_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

defense_dataset = CustomDataset(image_dir=output_dir, labels=train_labels, transform=defense_transform)
test_data = Subset(dataset, test_indices)

defense_loader = DataLoader(defense_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)  # No shuffle for test set

print(f"Defense dataset size: {len(defense_dataset)}")

Defense dataset size: 7552


In [2]:
import torch.nn as nn

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

# ConvMixer model with hard-coded parameters
def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10     # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [3]:
import torch

attacked_model_path = "/home/j597s263/scratch/j597s263/Models/ConvModels/Conv_Imagenette(attacked).mod"
model = torch.load(attacked_model_path, map_location="cuda", weights_only=False)
model = model.to("cuda")
model.eval()  

print("Attacked model loaded successfully!")

Attacked model loaded successfully!


In [4]:
attack_output_dir = "/home/j597s263/scratch/j597s263/Datasets/Attack/Imagenette"

attack_labels = [4] * len(os.listdir(attack_output_dir))  

attack_dataset = CustomDataset(image_dir=attack_output_dir, labels=attack_labels, transform=defense_transform)

attack_loader = DataLoader(attack_dataset, batch_size=32, shuffle=False)

print(f"Attack dataset size: {len(attack_dataset)}")

Attack dataset size: 945


In [5]:
# Hyperparameters
epochs = 10  
learning_rate = 0.001 
opt_eps = 1e-3
clip_grad = 1.0
device = 'cuda'

# Optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, eps=opt_eps)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=learning_rate,
    steps_per_epoch=len(defense_loader),
    epochs=epochs
)

criterion = nn.CrossEntropyLoss()

scaler = GradScaler()

# Training Loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in defense_loader:  # Use defense_loader for training
        images, labels = images.to(device), labels.to(device)

        with autocast(device_type='cuda'):
            outputs = model(images)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()

        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Training Loss on Defense Dataset: {running_loss/len(defense_loader):.4f}")

    # Testing phase on test_loader
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")

Epoch [1/10], Training Loss on Defense Dataset: 5.0085
Epoch [1/10], Test Loss: 2.6303, Test Accuracy: 17.51%
Epoch [2/10], Training Loss on Defense Dataset: 2.7232
Epoch [2/10], Test Loss: 2.2966, Test Accuracy: 14.88%
Epoch [3/10], Training Loss on Defense Dataset: 2.2831
Epoch [3/10], Test Loss: 2.2905, Test Accuracy: 12.16%
Epoch [4/10], Training Loss on Defense Dataset: 2.1631
Epoch [4/10], Test Loss: 2.3439, Test Accuracy: 13.21%
Epoch [5/10], Training Loss on Defense Dataset: 1.8803
Epoch [5/10], Test Loss: 2.5421, Test Accuracy: 11.22%
Epoch [6/10], Training Loss on Defense Dataset: 1.4865
Epoch [6/10], Test Loss: 2.6904, Test Accuracy: 12.58%
Epoch [7/10], Training Loss on Defense Dataset: 1.1102
Epoch [7/10], Test Loss: 2.8851, Test Accuracy: 13.00%
Epoch [8/10], Training Loss on Defense Dataset: 0.8298
Epoch [8/10], Test Loss: 3.0240, Test Accuracy: 12.16%
Epoch [9/10], Training Loss on Defense Dataset: 0.6769
Epoch [9/10], Test Loss: 3.0995, Test Accuracy: 12.79%


KeyboardInterrupt: 

In [ ]:
'''# Hyperparameters
epochs = 10  
learning_rate = 0.001  
opt_eps = 1e-3
clip_grad = 1.0
device = 'cuda'

# Optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, eps=opt_eps)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=learning_rate,
    steps_per_epoch=len(defense_loader),  
    epochs=epochs
)

# Loss function
criterion = nn.CrossEntropyLoss()

# Automatic Mixed Precision (AMP)
scaler = GradScaler()

# Training Loop
for epoch in range(epochs):
    # Training phase on defense dataset
    model.train()
    running_loss = 0.0

    for images, labels in defense_loader:  # Use defense_loader for training
        # Move data to GPU
        images, labels = images.to(device), labels.to(device)

        # Forward and backward pass with AMP
        with autocast(device_type='cuda'):
            outputs = model(images)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()

        # Gradient clipping
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        # Optimizer step
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item()

    # Log training loss for the epoch
    print(f"Epoch [{epoch+1}/{epochs}], Training Loss on Defense Dataset: {running_loss/len(defense_loader):.4f}")'''